In [ ]:
#default_exp models

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
#export
import numpy as np
from transformers import (AutoModelForSeq2SeqLM, AutoModelForSequenceClassification, AutoTokenizer)
from sentence_transformers import SentenceTransformer
from types import MethodType
from undecorated import undecorated
from travis_attack.config import Config

# Models

In [ ]:
#export
def _prepare_pp_tokenizer_and_model(cfg): 
    """As well as preparing the pp model and tokenizer this function also adds a new method `generate_with_grad` to 
    the pp model so that we can backprop when generating."""
    pp_tokenizer = AutoTokenizer.from_pretrained(cfg.pp_name)
    # takes about 3GB memory space up on the GPU
    # change the `local_files_only` argument if changing the model name 
    pp_model = AutoModelForSeq2SeqLM.from_pretrained(cfg.pp_name, local_files_only=True)
    generate_with_grad = undecorated(pp_model.generate)      # remove the @no_grad decorator from generate
    pp_model.generate_with_grad = MethodType(generate_with_grad, pp_model) 
    return pp_tokenizer, pp_model 
    
def _prepare_vm_tokenizer_and_model(cfg): 
    vm_tokenizer = AutoTokenizer.from_pretrained(cfg.vm_name)
    #TODO: do I need the .to(device) for the vm model when using accelerate?
    # change the `local_files_only` argument if changing the model name 
    vm_model = AutoModelForSequenceClassification.from_pretrained(cfg.vm_name, local_files_only=True).to(cfg.device)
    return vm_tokenizer, vm_model 
    
def _pad_model_token_embeddings(cfg, pp_model, vm_model, sts_model): 
    """Resize first/embedding layer of all models to be a multiple of cfg.embedding_padding_multiple. 
    Good for tensor core efficiency when using fp16.
    Makes changes to models in-place."""
    def pad_token_embeddings_to_multiple_of_n(model, n):
        def get_new_vocab_size(model): return int((np.floor(model.config.vocab_size / n) + 1) * n)
        model.resize_token_embeddings(get_new_vocab_size(model))
    pad_token_embeddings_to_multiple_of_n(pp_model, cfg.embedding_padding_multiple)
    pad_token_embeddings_to_multiple_of_n(vm_model, cfg.embedding_padding_multiple)
    # sts_model is from SentenceTransformers so needs a bit of unwrapping to access the base huggingface model 
    pad_token_embeddings_to_multiple_of_n(sts_model._first_module().auto_model, cfg.embedding_padding_multiple) 

def _update_config(cfg, vm_model, pp_model): 
    cfg.vm_num_labels = vm_model.num_labels
    cfg.vocab_size = pp_model.get_input_embeddings().num_embeddings   # unlike pp_tokenizer.vocab_size this includes the padding 
    return cfg
    
def prepare_models(cfg): 
    """Load tokenizers and models for vm, pp, sts. 
    Pad the first embedding layer if specified in the config.  
    Do layer freezing if specified (TODO). 
    Update config with some model-specific variables. 
    """
    vm_tokenizer, vm_model = _prepare_vm_tokenizer_and_model(cfg)
    pp_tokenizer, pp_model = _prepare_pp_tokenizer_and_model(cfg)
    sts_model = SentenceTransformer(cfg.sts_name)
    if cfg.pad_token_embeddings:  _pad_model_token_embeddings(cfg, pp_model, vm_model, sts_model)
    cfg = _update_config(cfg, vm_model, pp_model)
    return vm_tokenizer, vm_model, pp_tokenizer, pp_model, sts_model, cfg

## Usage

Externally we only need the `prepare_models(cfg)` function. This gives all the models and tokenizers needed for the other sections and also updates + returns the config. 

In [ ]:
cfg = Config()
vm_tokenizer, vm_model, pp_tokenizer, pp_model, sts_model, cfg = prepare_models(cfg)

#hide
## Export 

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 03_config.ipynb.
Converted 07_models.ipynb.
Converted 10_data.ipynb.
Converted 20_trainer.ipynb.
Converted 30_logging.ipynb.
Converted index.ipynb.
Converted run.ipynb.
